In [29]:
%gui qt
%matplotlib qt

import numpy as np
from matplotlib import pyplot as plt
from importlib import reload

import qcodes as qc
from qcodes import ParamSpec, new_data_set
from qcodes.dataset.database import initialise_database
from qcodes.dataset.data_set import DataSet
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.experiment_container import load_or_create_experiment

from plottr.data.datadict import DataDict
from plottr.data.qcodes_dataset import datadict_from_path_and_run_id
from plottr.apps.tools import make_sequential_flowchart

In [2]:
DBPATH = './test_qc_saveandload.db'

# Make 2D softsweep data

## define sweep

In [3]:
qc.config.core.db_location = DBPATH
initialise_database()
exp = load_or_create_experiment('2d_softsweep', sample_name='no sample')

x = np.linspace(0, 1., 11)
y = np.linspace(0, 1., 11)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = np.random.rand(*xx.shape)

def get_2dsoftsweep_results():
    for x, y, z in zip(xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)):
        yield dict(x=x, y=y, z=z)

## dummy measurement and retrieve as ddict

In [ ]:
ds = new_data_set('2d_softsweep', exp_id=exp.exp_id,
                  specs=[ParamSpec('x', 'numeric', unit='A'),
                         ParamSpec('y', 'numeric', unit='B'),
                         ParamSpec('z', 'numeric', unit='C', depends_on=['x', 'y']), ], )

results = get_2dsoftsweep_results()
for r in results:
    ds.add_result(r)
ds.mark_complete()

run_id = ds.run_id
ddict = datadict_from_path_and_run_id(DBPATH, run_id)

In [ ]:
# ddict

## Manual insertion and retrieval of updates

In [33]:
from plottr.data import qcodes_dataset; reload(qcodes_dataset)
from plottr.data.qcodes_dataset import QCodesDSLoader

### setting up the data
# ref_ddict = DataDict(x=dict(values=xx.reshape(-1)),
#                      y=dict(values=yy.reshape(-1)),
#                      z=dict(values=zz.reshape(-1), axes=['x', 'y']))

qc.config.core.db_location = DBPATH
initialise_database()
exp = load_or_create_experiment('2d_softsweep', sample_name='no sample')

_ds = new_data_set('2d_softsweep', exp_id=exp.exp_id,
                   specs=[ParamSpec('x', 'numeric', unit='A'),
                          ParamSpec('y', 'numeric', unit='B'),
                          ParamSpec('z', 'numeric', unit='C', depends_on=['x', 'y']), ], )

run_id = _ds.run_id
results = get_2dsoftsweep_results()

### setting up the flowchart
nodes, fc = make_sequential_flowchart([QCodesDSLoader])
loader = nodes[0]
loader.pathAndId = DBPATH, run_id

def check():
    nresults = _ds.number_of_results
    print(nresults)
    loader.update()
    ddict = fc.output()['dataOut']

    z_in = zz.reshape(-1)[:nresults]
    z_out = ddict.data_vals('z')
    assert z_in.size == z_out.size
    assert np.allclose(z_in, z_out, atol=1e-15)

### insert data in small chunks, and check
while True:
    try:
        ninsertions = np.random.randint(0, 5)
        for n in range(ninsertions):
            _ds.add_result(next(results))
    except StopIteration:
        _ds.mark_complete()
        break
    check()
check()

1
4
8
8
10
10
14
14
16
19
19
22
24
27
29
33
33
33
36
39
43
43
45
47
51
53
55
58
61
65
69
71
73
74
78
81
81
83
87
88
88
88
89
91
91
92
93
97
98
98
98
98
98
102
102
103
107
109
113
115
116
116
118
121
